## Transformer 概念 原理 实践 

### transformer 基本概念
transformer是一种基于自注意力的深度学习模型，用于处理序列数据。其中核心的组件有  
**自注意力机制**：允许模型在处理输入序列时，动态地关注序列中不同位置的信息，计算每个位置的加权表示  
**位置编码**: 由于Transformer不具备处理序列顺序的内置能力，因此引入位置编码来注入序列的位置信息  
**编码器和解码器结构**:原始的Transformer模型由编码器和解码器组成，编码器处理输入序列并生成上下文表示，解码器生成输出序列  

In [ ]:
"""
自注意力机制的理解

自注意力机制是说将用户的输入序列X通过不同的线性变化，映射为Q K V三个矩阵。
这三个矩阵的形状分别是N*D_K N*D_K  N*D_V 
其实在论文里面 这里D_K=D_V=D_MODEL/H 

其中：N就是序列的token长度； D_K 就是矩阵的列数；D_MODEL:就是模型嵌入的长度 H就是注意力头的数量

在计算的时候：Q@K_T 得到一个N*N的矩阵，其中每一行就代表了对应token在整个序列中的分数
矩阵经过SOFTMAX之后，就得到N个权重然后和V相乘，进行加权

详细可以参考：https://www.kimi.com/chat/19ac30a8-13e2-8d4d-8000-09c86faa98a0
            https://blog.csdn.net/qq_42691298/article/details/126590726
"""

In [ ]:
"""
位置编码的理解

位置编码就是用户首先输入的一个文本序列，比如猫坐在垫子上
然后token化之后得到得到一个token列表,比如[1,3,2,5,6]
对于每个token都会做成一个长度为d_model（嵌入维度）的向量
然后对这个token向量做位置编码的计算公式为
PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))

最后也会得到一个长度为d_model的位置向量。
然后原token向量+位置向量。就实现了对应token的位置信息注入

可视化理解就是
想象一个二维平面：
每个位置对应一个向量
相邻位置的向量相似但不相同
距离越远，向量差异越

这里位置编码其实还有一些变体，在这里咱叔不讨论
"""

In [ ]:
import numpy as np

def positional_encoding(seq_len, d_model):
    """
    生成位置编码矩阵
    seq_len: 序列长度
    d_model: 嵌入维度
    """
    pe = np.zeros((seq_len, d_model))
    
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            # 偶数维度使用 sin
            pe[pos, i] = np.sin(pos / (10000 ** (i / d_model)))
            # 奇数维度使用 cos
            if i + 1 < d_model:
                pe[pos, i + 1] = np.cos(pos / (10000 ** (i / d_model)))
    
    return pe

In [ ]:
"""
编码器和解码器的理解

以机器翻译为列：
编码器输入：I am a student    编码器输出： [seq_len,d_model]上下文矩阵
解码器输入: 我是一个学生        解码器输出： 在每个位置上输出一个概率分布，如下

位置0: 输入 <start>
       输出: [0.01, 0.85, 0.02, 0.01, ...]  ← 预测下一个词是"我"的概率最高

位置1: 输入 <start> 我
       输出: [0.02, 0.01, 0.90, 0.01, ...]  ← 预测下一个词是"是"的概率最高

位置2: 输入 <start> 我 是
       输出: [0.01, 0.01, 0.01, 0.88, ...]  ← 预测下一个词是"一"的概率最高

位置3: 输入 <start> 我 是 一
       输出: [0.01, 0.01, 0.01, 0.01, 0.87, ...]  ← 预测下一个词是"个"的概率最高

位置4: 输入 <start> 我 是 一 个
       输出: [0.01, 0.01, 0.01, 0.01, 0.01, 0.89, ...]  ← 预测下一个词是"学"的概率最高

位置5: 输入 <start> 我 是 一 个 学
       输出: [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.91, ...]  ← 预测下一个词是"生"的概率最高

位置6: 输入 <start> 我 是 一 个 学 生
       输出: [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.92, ...]  ← 预测下一个词是<end>的概率最高

计算损失：
位置0的预测 vs 真实标签"我"     → 计算损失
位置1的预测 vs 真实标签"是"     → 计算损失
位置2的预测 vs 真实标签"一"     → 计算损失
位置3的预测 vs 真实标签"个"     → 计算损失
位置4的预测 vs 真实标签"学"     → 计算损失
位置5的预测 vs 真实标签"生"     → 计算损失
位置6的预测 vs 真实标签"<end>" → 计算损失
"""

### 参考资料
[论文解读](https://neverbiasu.github.io/zh/posts/papers/transformer.html#%E6%A8%A1%E5%9E%8B%E5%90%AF%E5%8F%91%E4%B8%8E%E6%96%B9%E6%B3%95%E5%BB%B6%E4%BC%B8)
